In [1]:
import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
from sklearn.preprocessing import LabelEncoder
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib.image import imread
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

# Load the raw STAMP data

In [4]:
url = 'https://data.nist.gov/od/ds/mds2-2431/' # Raw data hosted here 
samples = pd.read_csv(url+'stamp_samples.csv') # Aliquot information
chemistry = pd.read_csv(url+'stamp_chemistry.csv') # Result of chemical tests

In [ ]:
chemistry

In [ ]:
samples

In [ ]:
len(samples['GUSAMPLEID'].unique()) # Total number of samples

In [ ]:
len(chemistry['analyte'].unique()) # Total number of unique analytes tested for

In [ ]:
len(samples['GUALIQUOTID'].unique()) # Total number of aliquots

In [ ]:
uaq = [len(samples[samples['GUSAMPLEID'] == s]['GUALIQUOTID'].unique()) for s in samples['GUSAMPLEID'].unique()]
print(np.min(uaq), np.max(uaq))

# Plot a summary

In [ ]:
# Plot the predominant species at each colony.

# Get location of each colony
lats = {c:set() for c in samples['COLONY_NAME'].unique()}
lons = {c:set() for c in samples['COLONY_NAME'].unique()}
for c in samples['COLONY_NAME'].unique():
    lats[c] = lats[c].union(set(samples[samples['COLONY_NAME'] == c]['LATITUDE']))
    lons[c] = lons[c].union(samples[samples['COLONY_NAME'] == c]['LONGITUDE'])

# St. Lawrence  Isl. and St. George Isl. each have 3 slightly different sample locations, but are 
# essentially the same values so just average them for this purpose.
locs = {c:(np.mean(list(lats[c])), np.mean(list(lons[c]))) for c in lats.keys()}

plt.figure(figsize=(5,10))

global_ = np.array([locs[c] for c in locs])
source_proj = ccrs.PlateCarree()
ax = plt.subplot(1,1,1, projection=source_proj)

ax.stock_img()
ax.set_extent((np.min(global_[:,1])-np.std(global_[:,1])*0.3, 
                   np.max(global_[:,1])+np.std(global_[:,1])*0.3,
                   np.min(global_[:,0])-np.std(global_[:,0])*0.3, 
                   np.max(global_[:,0])+np.std(global_[:,0])*0.3)
                 )
    
enc = LabelEncoder()
enc.fit(np.unique(samples['COMMON_NAME']))

for colony_, loc_ in locs.items():
    top_bird = sorted(zip(*np.unique(samples[samples['COLONY_NAME'] == colony_]['COMMON_NAME'], return_counts=True)),
           key=lambda x:x[1],
           reverse=True
          )[0][0]
    color = enc.transform([top_bird])[0]
    ax.plot(loc_[1], loc_[0], color='C{}'.format(color), linewidth=0, marker='o')

custom_lines = [Line2D([0], [0], color='C{}'.format(enc.transform([x])[0]), lw=4) 
               for x in enc.classes_]
ax.legend(custom_lines, [x for x in enc.classes_], 
         bbox_to_anchor=(.8, .5))
_ = ax.set_title('Predominant Species')

In [ ]:
chart = sns.catplot(y='GEOGRAPHIC_AREA_STAMP', kind="count",
            palette="pastel", edgecolor=".6",
            data=samples.drop_duplicates(subset='GUSAMPLEID', keep='first')
                   )
chart.set_ylabels('Geographic Area')
chart.set_xlabels('Count')

In [ ]:
chart = sns.catplot(y='COLLECTION_YEAR', kind="count",
            palette="pastel", edgecolor=".6",
            data=samples.drop_duplicates(subset='GUSAMPLEID', keep='first'),
            order=np.arange(1999, 2010+1),
            )
chart.set_ylabels('Collection Year')
chart.set_xlabels('Count')

In [ ]:
s = samples.drop_duplicates(subset='GUSAMPLEID', keep='first')
y = s['COMMON_NAME'] 
order = sorted(y.unique(), key=lambda x:x.split(' ')[1])

fig,ax = plt.subplots(nrows=1, ncols=1,figsize=(5,7))
ax.bar(
    x=order,
    height=[np.sum(y==c) for c in order],
)
plt.ylabel('Number of Observations', fontsize=12)
_ = plt.xticks(rotation=90, fontsize=12)
_ = plt.yticks(fontsize=12)

for i,c in enumerate(order):
    if (i < 3 or i > 4):
        shift = .16
    else:
        shift=.2
    plt.text(i-shift,np.sum(y==c)-10, str(np.sum(y==c)), color='white')

# Revise the data

## Merge chemistry and samples into a dataset

In [ ]:
# Check which analytes co-eluted and need to be merged

def custom_parser(to_merge):
    def get(full_name, analyte_name):
        multiples = [x.strip('and ') for x in full_name.split(',')] # No commas used in chemical name must be guaranteed
        if (len(multiples) == 1): # Didn't split out any commas so just a pair
            multiples = full_name.split(' and ')
        return [' '.join([analyte_name, x.split(' ')[-1]]) for x in multiples]
        
    columns = []
    for m in to_merge:
        if m.startswith('BDEs'):
            columns.append(get(m, 'BDE'))
        elif m.startswith('PCBs'):
            columns.append(get(m, 'PCB'))
        else:
            columns.append(m.split(' and '))
    
    return columns

analytes_available = set(chemistry['analyte'].unique())
to_merge = [x for x in analytes_available if 'and' in x] # Columns that need to be merged
parsed = custom_parser(to_merge) # Individual columns split apart

In [ ]:
import random
from itertools import chain
random.seed(0)

# Can't merge on GUSAMPLEID because it doesn't exist in chemistry
merged_inner = pd.merge(left=chemistry, 
                        right=samples, 
                        left_on='GUALIQUOTID', 
                        right_on='GUALIQUOTID')

# Sanity check that individual aliquots come from same sample
for al_id in merged_inner['GUALIQUOTID'].unique(): 
    assert(len(merged_inner[merged_inner['GUALIQUOTID'] == al_id]['GUSAMPLEID'].unique()) == 1)
    
X,y = None, None

for i,sa_id in tqdm.tqdm(enumerate(merged_inner['GUSAMPLEID'].unique())):
    # For each sample ID, find all aliquots and combine results
    this_sample = merged_inner[merged_inner['GUSAMPLEID'] == sa_id]
    frames = []
    for al_id in this_sample['GUALIQUOTID'].unique():
        frames.append(this_sample[this_sample['GUALIQUOTID'] == al_id])
    combined = pd.concat(frames, axis=0, sort=True, verify_integrity=True, ignore_index=True)
    
    # Check analytes not duplicated
    u_, c_ = np.unique(combined['analyte'], return_counts=True)
    if (np.any(c_ > 1)):
        print('duplicates found for {} {} {}'.format(i, sa_id, combined['COMMON_NAME'].iloc[0]))
        combined.drop_duplicates(subset=['analyte'], keep='first', inplace=True)

    # Merge, impute, etc.
    for merged, aset in zip(to_merge, parsed): 
        if (merged not in combined['analyte'].values):
            # Sum was not directly tested, try to construct it
            isum = 0.0
            rand = 0.0
            for analyte in aset:
                if analyte in combined['analyte'].values:
                    v = combined[combined['analyte'] == analyte]['value'].values[0]
                    isum += (0.0 if np.isnan(v) else v) # Non-detects impute to 0, according to Jared
                    if (np.isnan(v)):
                        assert(combined[combined['analyte'] == analyte]['dl'].values[0] > 0), 'check dL are > 0'
                        rand += (random.random()*combined[combined['analyte'] == analyte]['dl'].values[0]) # dL should be present if value is not, so this is only a float if all measurements were NaN (below dL)
                else:
                    isum = np.nan # Wasn't even measured, can't build sum
                    break
            
            if isum == 0.0: # If all were tested but undetected 
                new_row = pd.DataFrame([[np.nan] * combined.shape[1]], 
                                       columns=combined.columns)
                new_row['value'] = rand
                new_row['analyte'] = merged
                combined = pd.concat([combined, new_row], ignore_index=True)
            else: 
                # If some are measured, report net as sum of those detected
                # In the case that one or more terms missing this becomes NaN (see above) as we cannot construct this some
                new_row = pd.DataFrame([[np.nan] * combined.shape[1]], 
                                       columns=combined.columns)
                new_row['value'] = isum
                new_row['analyte'] = merged
                combined = pd.concat([combined, new_row], ignore_index=True)
        else:
            # Sum was directly tested for
            if np.isnan(combined[combined['analyte'] == merged]['value'].values[0]):
                # Below dL, impute to 0 < RNG < dL
                assert(combined[combined['analyte'] == merged]['dl'].values[0] > 0), 'check dL are > 0'
                v = random.random()*combined[combined['analyte'] == merged]['dl'].values[0]
                combined.loc[combined[combined['analyte'] == merged].index, 'value'] = v
            else:
                # Above dL, nothing to do
                pass
            
        # Remove individual terms
        for analyte in aset:
            combined = combined.drop(index=combined[combined['analyte'] == analyte].index)
        
    # For non-detects that are not going to be summed, impute to 0 < RNG < dL
    # Skip analytes that were just merged and those which had to be removed
    for analyte in [a for a in analytes_available if a not in to_merge+list(chain.from_iterable(parsed))]: 
        if analyte in combined['analyte'].values:
            # Analyte was tested for - if below dL, impute 0 < RNG < dL
            v = combined[combined['analyte'] == analyte]['value'].values[0]
            if np.isnan(v):
                assert(combined[combined['analyte'] == analyte]['dl'].values[0] > 0), 'check dL are > 0'
                v = random.random()*combined[combined['analyte'] == analyte]['dl']
                combined.loc[combined[combined['analyte'] == analyte].index, 'value'] = v
        else:
            # Have to add a NaN (not tested) for this analyte
            new_row = pd.DataFrame([[np.nan] * combined.shape[1]], 
                                       columns=combined.columns)
            new_row['analyte'] = analyte
            combined = pd.concat([combined, new_row], ignore_index=True)

    # 2. Store targets
    target_df = combined[['COLONY_NAME', 'COLLECTION_YEAR', 'COMMON_NAME', 'GUSAMPLEID']]
    for t in target_df.columns: # Assert that each data point has only one name, year, etc.
        assert(target_df[t].nunique() == 1), '{} {}'.format(t, target_df[t].nunique())
        
    # 3. Convert first row ('analyte') into header and remove as row
    raw = combined[['analyte', 'value']].transpose()
    raw.columns = raw.iloc[0]
    raw = raw.iloc[pd.RangeIndex(len(raw)).drop(0)]
        
    X = pd.concat([X, raw], axis=0, sort=True, verify_integrity=True,  
                    ignore_index=True
                    ) 
    y = pd.concat([y, target_df[:1]], axis=0, sort=True, verify_integrity=True,
                    ignore_index=True)
    
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [ ]:
X.shape

## Set minimum observation threshold to 95%

In [ ]:
threshold = 0.95 # Set that analytes must be measured at least this % of the time
chosen = X.columns[X.isnull().sum() < X.shape[0]*(1.0-threshold)]

In [ ]:
data = X[chosen].join(y, how='outer').dropna() # Remove NaN entries since they correspond to test not performeed

In [ ]:
used = data[data['COMMON_NAME'] != 'Unknown murre']

In [ ]:
used

Note that these data will vary slightly relative to that used due to the generation of random numbers to
interpolate below detection limits.  Even if the random number generator's seed is set, the use of 
unorder sets in the above loop means things may be accessed or ordered at certain stages in a machine-dependent manner.

This overall results are insensitive to this randomness since these generated numbers only contributed to things below the detection limit, and had only marginal quantitative impact, and no qualitative impact, on any of the results.

# Unsupervised analysis

In [ ]:
features = [c for c in used.columns if c not in {"COLLECTION_YEAR", "COLONY_NAME", "COMMON_NAME", "GUSAMPLEID"}]

In [ ]:
pca = PCA(n_components=2)
ss = StandardScaler()
X_proj = pca.fit_transform(ss.fit_transform(used[features]))

plt.figure(figsize=(5,5))
for bird in np.unique(y['COMMON_NAME']):
    mask = used['COMMON_NAME'] == bird
    plt.plot(X_proj[mask,0], X_proj[mask,1], 'o', label=bird)
plt.legend(loc='best')
plt.xlabel('PC 1 ({}%)'.format('%.1f'%(100*pca.explained_variance_ratio_[0])))
plt.ylabel('PC 2 ({}%)'.format('%.1f'%(100*pca.explained_variance_ratio_[1])))

In [ ]:
%matplotlib notebook

pca = PCA(n_components=3)
ss = StandardScaler()
X_proj = pca.fit_transform(ss.fit_transform(used[features]))

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')
for bird in np.unique(used['COMMON_NAME']):
    mask = used['COMMON_NAME'] == bird 
    ax.scatter(X_proj[mask,0], X_proj[mask,1], X_proj[mask,2], label=bird)
ax.set_xlabel('PC 1 ({}%)'.format('%.1f'%(100*pca.explained_variance_ratio_[0])))
ax.set_ylabel('PC 2 ({}%)'.format('%.1f'%(100*pca.explained_variance_ratio_[1])))
ax.set_zlabel('PC 3 ({}%)'.format('%.1f'%(100*pca.explained_variance_ratio_[2])))
ax.view_init(elev=20., azim=-45)

In [ ]:
%matplotlib inline

pca_comps = sorted(zip(pca.components_[0], features), key=lambda x:np.abs(x[0]), reverse=True)

plt.figure(figsize=(10,5.5))
plt.bar(
    x=[a_[1] for a_ in pca_comps],
    height=np.abs([a_[0] for a_ in pca_comps]),
)
_ = plt.xticks(rotation=90)
plt.ylabel('|Coefficient| in PC 1')